In [ ]:
import sys
import os
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

from components import rag
import pandas as pd


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.91it/s]
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Answer: Civil servants in Malaysia have the option to choose flexible working hours during Ramadan, allowing them to leave work 30 minutes earlier.
Reference1: Civil servants can opt for flexible working hours during Ramadan, says Fahmi.
Reference2: Selangor civil servants allowed to leave work 30 minutes early during Ramadan.
Reference3: Flexi work hours for Perlis civil servants during Ramadan.


In [ ]:
df = pd.read_csv("TestQuestion20.csv")
df_1K = pd.read_csv("../data/1K_news.csv")

if 'Question' not in df.columns:
    raise ValueError("The CSV file must contain a 'Question' column.")

answers = []
retrieved_docs_id = []
retrieved_docs1 = []
retrieved_docs2 = []
retrieved_docs3 = []

document_dataset = "../data/1K_news.csv"

for question in df['Question']:

    output = rag.query_answering_system(question, document_dataset)

    answers.append(output['answer'])
    retrieved_docs_id.append(str(output['retrieved_docs_id']))

    retrieved_docs1.append(
        df_1K.loc[df_1K['content_id'] == output['retrieved_docs_id'][0], 'text'].values[0]
    )
    retrieved_docs2.append(
        df_1K.loc[df_1K['content_id'] == output['retrieved_docs_id'][1], 'text'].values[0]
    )
    retrieved_docs3.append(
        df_1K.loc[df_1K['content_id'] == output['retrieved_docs_id'][2], 'text'].values[0]
    )
    

Batches: 100%|██████████| 1/1 [00:00<00:00, 93.16it/s]
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Batches: 100%|██████████| 1/1 [00:00<00:00, 87.68it/s]
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Batches: 100%|██████████| 1/1 [00:00<00:00, 82.16it/s]
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Batches: 100%|██████████| 1/1 [00:00<00:00, 30.21it/s]
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Batches: 100%|██████████| 1/1 [00:00<00:00, 86.39it/s]
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Batches: 100%|██████████| 1/1 [00:00<00:00, 49.67it/s]
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Batches: 100%|██████████| 1/1 [00:00<00:00, 71.36it/s]
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Batches: 100%|██████████| 1/1 [00:00<00:00, 19.11it/s]
Setting `pad_token_id` to `eos_token_id`:151643 f

In [15]:
df['answer'] = answers
df['retrieved_docs_id'] = retrieved_docs_id
df['retrieved document1'] = retrieved_docs1
df['retrieved document2'] = retrieved_docs2
df['retrieved document3'] = retrieved_docs3

df.to_csv("TestQuestion20_with_answers.csv", index=False)

print("Processing complete. The output file is saved as 'TestQuestion20_with_answers.csv'.")

Processing complete. The output file is saved as 'TestQuestion20_with_answers.csv'.
